In [ ]:
import json
import pandas as pd
import networkx as nx
import seaborn as sns

In [ ]:
%matplotlib inline

# LN data

data_file = "../LNdata/0314.json"

In [ ]:
node_keys = ["pub_key","last_update"]#,"is_reachable"]

In [ ]:
edge_keys = ["node1_pub","node2_pub","last_update","capacity","channel_id"]

In [ ]:
data_file = "../LNdata/lncaptures/lngraph/2019/1549032366.json"

In [ ]:
with open(data_file) as f:
    ln_json = json.load(f)

In [ ]:
ln_json.keys()

In [ ]:
ln_json["nodes"][0]

In [ ]:
ln_json["edges"][0]

## ???

ln_json["chains"]

ln_json["testnet"]

# LN Network

## nodes

In [ ]:
nodes = pd.DataFrame(ln_json["nodes"])[node_keys]
len(nodes)

In [ ]:
nodes = nodes[nodes["last_update"] > 0]
len(nodes)

In [ ]:
nodes.head()

nodes["is_reachable"].value_counts()

In [ ]:
nodes["last_update"].hist(bins=50)

## edges

In [ ]:
edges = pd.DataFrame(ln_json["edges"])[edge_keys]
len(edges)

In [ ]:
edges.head()

## NOT TOO MANY capacity changes :(

In [ ]:
edges.groupby(["node1_pub","node2_pub"])["capacity"].nunique().value_counts()

In [ ]:
edges["capacity"].value_counts()[:10]

## There are node pairs that have several channels between them!

In [ ]:
edges.groupby(["node1_pub","node2_pub"])["channel_id"].nunique().value_counts()

In [ ]:
edges.groupby(["node1_pub","node2_pub","channel_id","capacity"])["last_update"].nunique().value_counts()

## Direction - NO BIDIRECTIONALITY IN THE NETWORK!!!

In [ ]:
node_pairs = edges[["node1_pub","node2_pub"]].drop_duplicates()
node_pairs.shape

In [ ]:
node_pairs_reverse = node_pairs[["node2_pub","node1_pub"]]
node_pairs_reverse.columns = ["node1_pub","node2_pub"]

In [ ]:
node_pairs.merge(node_pairs_reverse, on=["node1_pub","node2_pub"]).shape

## Build graph

In [ ]:
G = nx.from_pandas_edgelist(edges, "node1_pub", "node2_pub", ["last_update","capacity"])

In [ ]:
print(
"Nodes:", G.number_of_nodes(),
"\nEdges:", G.number_of_edges(),
"\nLoops:", G.number_of_selfloops(),
"\nConnected components:", nx.number_connected_components(G),
)

In [ ]:
gcc = max(nx.connected_component_subgraphs(G), key=len)

In [ ]:
print(
"Nodes:", gcc.number_of_nodes(),
"\nEdges:", gcc.number_of_edges(),
#"\nDiameter:", nx.diameter(gcc)
)

In [ ]:
degrees = dict(nx.degree(G))

In [ ]:
nodes["degree"] = nodes["pub_key"].apply(lambda x: degrees.get(x,0))

### Latest nodes

In [ ]:
(nodes["last_update"].max() - nodes["last_update"].min()) // 86400

In [ ]:
max_time = nodes["last_update"].max()

In [ ]:
recent_nodes = nodes[nodes["last_update"] > max_time - 7*86400]

In [ ]:
len(recent_nodes)

In [ ]:
sns.jointplot(data=recent_nodes, x="last_update", y="degree")